In [1]:
import pandas as pd
import numpy as np

In [2]:
file_name='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/coursera/project/kc_house_data_NaN.csv'
df=pd.read_csv(file_name)

In [3]:
df.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,7129300520,20141013T000000,221900.0,3.0,1.00,1180,5650,1.0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,6414100192,20141209T000000,538000.0,3.0,2.25,2570,7242,2.0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,5631500400,20150225T000000,180000.0,2.0,1.00,770,10000,1.0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,3,2487200875,20141209T000000,604000.0,4.0,3.00,1960,5000,1.0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,4,1954400510,20150218T000000,510000.0,3.0,2.00,1680,8080,1.0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
house_df = df.drop(['id','Unnamed: 0',"date","lat","long"], axis = 1)

print(house_df.shape)

(21613, 17)


In [5]:
house_df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
0,221900.0,3.0,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,1340,5650
1,538000.0,3.0,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,1690,7639
2,180000.0,2.0,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,2720,8062
3,604000.0,4.0,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,1360,5000
4,510000.0,3.0,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,1800,7503


In [6]:
house_df.isna().sum()

price             0
bedrooms         13
bathrooms        10
sqft_living       0
sqft_lot          0
floors            0
waterfront        0
view              0
condition         0
grade             0
sqft_above        0
sqft_basement     0
yr_built          0
yr_renovated      0
zipcode           0
sqft_living15     0
sqft_lot15        0
dtype: int64

In [7]:
house_df =house_df.dropna()

In [8]:
house_df.isna().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [9]:
array = house_df.values

In [10]:
X = array[:,1:]
Y = array[:,0]

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3,random_state=2222)

In [12]:
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing data
X_test_norm = norm.transform(X_test)

### Defining Regression models

In [13]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=7, shuffle=True, random_state=2222) #set 10-fold cross validation after shuffle the dataset with random seed 7

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR


# define a knn model with default parameter setting
knn = KNeighborsRegressor()
# run the previously defined 10-fold validation on the dataset
results = cross_val_score(knn, X_train_norm, Y_train, cv=kfold)
# print the averae r squared scores
print("Average R2 of KNN:", results.mean())

lr = LinearRegression()
results = cross_val_score(lr, X_train_norm, Y_train, cv=kfold)
print("Average R2 of LR:",results.mean())

svr = SVR()
results = cross_val_score(svr, X_train_norm, Y_train, cv=kfold)
print("Average R2 of SVM:",results.mean())

Average R2 of KNN: 0.6907500934735976
Average R2 of LR: 0.6431351137623648
Average R2 of SVM: -0.057572962395279044


In [16]:
# fine tune parameters for knn model
from sklearn.model_selection import GridSearchCV

grid_params_knn = {
    'n_neighbors': [3, 5, 7, 9, 11, 19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan','chebyshev','minkowski'],
}

knn = KNeighborsRegressor()
gs_knn_result = GridSearchCV(knn, grid_params_knn, cv=kfold).fit(X_train_norm, Y_train)
print(gs_knn_result.best_score_)

0.725957809114791


In [17]:
# use the best model and evaluate on testing set
gs_knn_result.best_estimator_.score(X_test_norm, Y_test)

0.7498791773981517

In [18]:
# check the parameter setting for the best selected model
gs_knn_result.best_params_

{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}

In [19]:
# fine tune parameters for lr model
from sklearn.model_selection import GridSearchCV

grid_params_lr = {
    'fit_intercept': [True, False],
    'normalize': [True, False]
}

lr = LinearRegression()
gs_lr_result = GridSearchCV(lr, grid_params_lr, cv=kfold).fit(X_train_norm, Y_train)
print(gs_lr_result.best_score_)

0.6431351137623648


In [20]:
# use the best model and evaluate on testing set
gs_lr_result.best_estimator_.score(X_test_norm, Y_test)

0.6650147602100207